# Grab Dimensions Data

https://app.dimensions.ai/discover/publication

This notebook handles the downloads of Dimensions data.


In [ ]:
import pandas as pd
import numpy as np

## Load Journals

In [ ]:
origin_folder = 'Journals_Articles'

In [ ]:
journals_file = f'{origin_folder}/ABS期刊汇总_DimentionsSearch.csv'
journals = pd.read_csv(journals_file, usecols=['search title', 'search id'])
journals = journals.dropna()

In [ ]:
len(journals)

In [ ]:
journals_index = '1-10'

In [ ]:
start = journals_index.split('-')[0]
end = journals_index.split('-')[1]
journals = journals[start - 1:end]

## Request Functions

This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [ ]:
import requests
import json

from requests import ConnectionError, ReadTimeout

def grab_from_url_content(url):
    headers = {'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }
    rescontent = ''
    try:
        res = requests.get(url, headers=headers, timeout=10)
        rescontent = res.text
    except ConnectionError as ce:
        print('ConnectionError: ' + str(ce))
        return grab_from_url_content(url)
    except ReadTimeout as rte:
        print('ReadTimeout: ' + str(rte))
        return grab_from_url_content(url)

    return rescontent



## Parser

We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [ ]:
from html.parser import HTMLParser

class DimensionsHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_next_trigger = False
    next_trigger_url = ''

    def handle_starttag(self, tag, attrs):
        if tag == 'article':
            self.in_article = True
        
        if self.in_article:
            for attr in attrs:
                if attr[0] == 'data-doc':
                    self.articles.append(json.loads(attr[1]))
                    break

        if tag == 'a':
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'nextPage-trigger'):
                    self.is_next_trigger = True
                if self.is_next_trigger and (attr[0] == 'href'):
                    self.next_trigger_url = f'https://app.dimensions.ai{attr[1]}&{search_params}'
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_next_trigger = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = DimensionsHTMLParser()

## Grab Data

The data fetching starts from here.

In [ ]:
%%time
for row in journals.iterrows():
    journal_search = row['search id']
    # This is very important for data fetching, generally you get url in the form as 'https://app.dimensions.ai/discover/publication?and_facet_for=2209&and_facet_for=2202&or_facet_source_title=jour.1082997', and search_params is just the substring after '?'.
    # Here is where we need to change for each search.
    search_params = f'or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title={journal_search}'
    grab_url = f'https://app.dimensions.ai/discover/publication?{search_params}'

    parser.feed(grab_from_url_content(grab_url))
    next_trigger_anchor = ''
    i = 0
    while parser.next_trigger_url != next_trigger_anchor:
        if i % 100 == 0:
            print(str(i))
        i+=1
        next_trigger_anchor = parser.next_trigger_url
        parser.feed(grab_from_url_content(parser.next_trigger_url))

In [ ]:
print(len(parser.articles))

In [ ]:
with open(f"{origin_folder}/articles_{journals_index}.json", "w") as dump_f:
        dump_f.write(json.dumps(parser.articles))

In [ ]:
with open(f'{origin_folder}/articles_{journals_index}.json') as f:
    articles = json.load(f)

df_articles = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])

for article in articles:
    if 'doi' not in article:
        continue
    if article['doi'] in df_articles['DI']:
        continue
    df_article = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])
    df_article['SO'] = [article['source_title']]
    df_article['SO_id'] = [article['source_title_id']]
    df_article['DI'] = [article['doi']]
    df_article['dimensions_id'] = [article['id']]
    df_articles = df_articles.append(df_article, ignore_index=True)

df_articles.to_csv(f'{origin_folder}/articles_{journals_index}.csv', index=False)